Now here's my first deep neural net (multilayer perceptron), I'll use the [MNIST][1] dataset.

[1]:https://en.wikipedia.org/wiki/MNIST_database

In [1]:
import tensorflow as tf
import numpy as np
#first get the dataset

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")
print(X_train)
print(y_train)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[7 3 4 ..., 5 6 8]


In [2]:
#visualization
print(X_train.shape)

(55000, 784)


With that we can see that the mnist dataset is conformed by 55000 images each one containing 784 pixels having a square shape of 28 pixels * 28 pixels

In [3]:
#defining our neural net
n_inputs = 784  # MNIST data input (img shape: 28*28)
n_hidden1 = 300 # number of features(neurons) in this layer
n_hidden2 = 100 # number of fatures(neurons) in layer 2
n_outputs = 10 # these are our classes (0-9 digits)

In [4]:
# Placeholders for inputs and outputs
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [5]:
def neuron_layer(X, n_neurons, name, activation=None): 
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev) 
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation=="relu":
            return tf.nn.relu(z) 
        else:
            return z

In [6]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits = neuron_layer(hidden2, n_outputs, "outputs")

In [7]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels= y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [8]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [11]:
n_epochs = 47
n_batches = 100
batch_size = 50

In [12]:
with tf.Session() as sess: 
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size )
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "first_neural_net.ckpt")

0 Train accuracy: 0.96 Test accuracy: 0.9107
1 Train accuracy: 0.96 Test accuracy: 0.9341
2 Train accuracy: 0.98 Test accuracy: 0.9413
3 Train accuracy: 0.92 Test accuracy: 0.947
4 Train accuracy: 0.96 Test accuracy: 0.9509
5 Train accuracy: 0.94 Test accuracy: 0.9548
6 Train accuracy: 0.98 Test accuracy: 0.9587
7 Train accuracy: 0.94 Test accuracy: 0.9595
8 Train accuracy: 1.0 Test accuracy: 0.9609
9 Train accuracy: 0.96 Test accuracy: 0.9648
10 Train accuracy: 1.0 Test accuracy: 0.9658
11 Train accuracy: 0.98 Test accuracy: 0.9662
12 Train accuracy: 1.0 Test accuracy: 0.9674
13 Train accuracy: 1.0 Test accuracy: 0.9674
14 Train accuracy: 0.98 Test accuracy: 0.969
15 Train accuracy: 1.0 Test accuracy: 0.97
16 Train accuracy: 1.0 Test accuracy: 0.9714
17 Train accuracy: 1.0 Test accuracy: 0.9708
18 Train accuracy: 0.96 Test accuracy: 0.9711
19 Train accuracy: 0.96 Test accuracy: 0.9713
20 Train accuracy: 1.0 Test accuracy: 0.973
21 Train accuracy: 1.0 Test accuracy: 0.9724
22 Train acc

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "my_model_final.ckpt")
    X_new_scaled = [...] # some new images (scaled from 0 to 1) 
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)
